In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,2020-11-26T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1545,253,1798,36710,...,188,1768,18890,1371,35885.0,22884.0,58769,922944,632415.0,NaN
5813,2020-11-26T17:00:00,ITA,9,Toscana,43.769231,11.255889,1712,281,1993,46142,...,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0,effettuati 75027 tamponi rapidi antigenici di...
5814,2020-11-26T17:00:00,ITA,10,Umbria,43.106758,12.388247,349,68,417,9017,...,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0,NaN
5815,2020-11-26T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,128,17,145,1468,...,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,2020-11-26T17:00:00,19,Sicilia,1545,253,1798,36710,38508,188,1768,18890,1371,35885.0,22884.0,58769,922944,632415.0
5813,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0
5814,2020-11-26T17:00:00,10,Umbria,349,68,417,9017,9434,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0
5815,2020-11-26T17:00:00,2,Valle d'Aosta,128,17,145,1468,1613,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-26', '2020-11-25')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-26T17:00:00,1,Piemonte,4992,403,5395,73011,78406,485,2751,74723,5875,67389.0,91615.0,159004,1490715,933365.0
1,2020-11-26T17:00:00,2,Valle d'Aosta,128,17,145,1468,1613,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0
2,2020-11-26T17:00:00,3,Lombardia,7996,934,8930,131471,140401,2372,5697,231042,21212,290349.0,102306.0,392655,3949040,2304509.0
3,2020-11-26T17:00:00,5,Veneto,2447,304,2751,72401,75152,14,3980,55403,3501,34933.0,99123.0,134056,2734036,1045655.0
4,2020-11-26T17:00:00,6,Friuli Venezia Giulia,599,55,654,13168,13822,286,814,12536,741,23010.0,4089.0,27099,687612,270753.0
5,2020-11-26T17:00:00,7,Liguria,1075,114,1189,12303,13492,-563,570,34137,2322,34280.0,15671.0,49951,584154,285519.0
6,2020-11-26T17:00:00,8,Emilia-Romagna,2679,258,2937,68535,71472,-1054,2157,37835,5548,71624.0,43231.0,114855,2057518,1055046.0
7,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0
8,2020-11-26T17:00:00,10,Umbria,349,68,417,9017,9434,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0
9,2020-11-26T17:00:00,11,Marche,586,90,676,15733,16409,378,519,10443,1221,21645.0,6428.0,28073,401261,233374.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-25T17:00:00,1,Piemonte,5095,403,5498,72423,77921,846,2878,72529,5803,66552.0,89701.0,156253,1468297,923880.0
1,2020-11-25T17:00:00,2,Valle d'Aosta,137,16,153,1530,1683,-84,27,4219,297,5526.0,673.0,6199,56803,32389.0
2,2020-11-25T17:00:00,3,Lombardia,8114,942,9056,128973,138029,-10731,5173,227924,21005,286833.0,100125.0,386958,3904809,2289221.0
3,2020-11-25T17:00:00,5,Veneto,2422,305,2727,72411,75138,1784,2660,51509,3429,31612.0,98464.0,130076,2714890,1039770.0
4,2020-11-25T17:00:00,6,Friuli Venezia Giulia,588,54,642,12894,13536,189,696,12034,715,22310.0,3975.0,26285,679986,268850.0
5,2020-11-25T17:00:00,7,Liguria,1115,123,1238,12817,14055,-728,460,33018,2308,33844.0,15537.0,49381,578449,283560.0
6,2020-11-25T17:00:00,8,Emilia-Romagna,2763,249,3012,69514,72526,1182,2130,34681,5493,70428.0,42272.0,112700,2040254,1048727.0
7,2020-11-25T17:00:00,9,Toscana,1754,286,2040,47240,49280,-1426,986,46318,2378,75858.0,22118.0,97976,1494998,928848.0
8,2020-11-25T17:00:00,10,Umbria,361,70,431,9506,9937,-260,386,12186,355,6314.0,16164.0,22478,399810,219242.0
9,2020-11-25T17:00:00,11,Marche,578,94,672,15359,16031,314,448,10312,1211,21196.0,6358.0,27554,398356,231259.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-26T17:00:00,1,Piemonte,4992,403,5395,73011,78406,485,2751,74723,5875,67389.0,91615.0,159004,1490715,933365.0
1,2020-11-26T17:00:00,2,Valle d'Aosta,128,17,145,1468,1613,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0
2,2020-11-26T17:00:00,3,Lombardia,7996,934,8930,131471,140401,2372,5697,231042,21212,290349.0,102306.0,392655,3949040,2304509.0
3,2020-11-26T17:00:00,4,Trentino-Alto Adige,851,83,934,12878,13812,-235,589,22161,1124,29832.0,7265.0,37097,666100,277978.0
4,2020-11-26T17:00:00,5,Veneto,2447,304,2751,72401,75152,14,3980,55403,3501,34933.0,99123.0,134056,2734036,1045655.0
5,2020-11-26T17:00:00,6,Friuli Venezia Giulia,599,55,654,13168,13822,286,814,12536,741,23010.0,4089.0,27099,687612,270753.0
6,2020-11-26T17:00:00,7,Liguria,1075,114,1189,12303,13492,-563,570,34137,2322,34280.0,15671.0,49951,584154,285519.0
7,2020-11-26T17:00:00,8,Emilia-Romagna,2679,258,2937,68535,71472,-1054,2157,37835,5548,71624.0,43231.0,114855,2057518,1055046.0
8,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0
9,2020-11-26T17:00:00,10,Umbria,349,68,417,9017,9434,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-25T17:00:00,1,Piemonte,5095,403,5498,72423,77921,846,2878,72529,5803,66552.0,89701.0,156253,1468297,923880.0
1,1,2020-11-25T17:00:00,2,Valle d'Aosta,137,16,153,1530,1683,-84,27,4219,297,5526.0,673.0,6199,56803,32389.0
2,2,2020-11-25T17:00:00,3,Lombardia,8114,942,9056,128973,138029,-10731,5173,227924,21005,286833.0,100125.0,386958,3904809,2289221.0
3,19,2020-11-25T17:00:00,4,Trentino-Alto Adige,863,82,945,13102,14047,-81,520,21357,1104,29444.0,7064.0,36508,659675,276171.0
4,3,2020-11-25T17:00:00,5,Veneto,2422,305,2727,72411,75138,1784,2660,51509,3429,31612.0,98464.0,130076,2714890,1039770.0
5,4,2020-11-25T17:00:00,6,Friuli Venezia Giulia,588,54,642,12894,13536,189,696,12034,715,22310.0,3975.0,26285,679986,268850.0
6,5,2020-11-25T17:00:00,7,Liguria,1115,123,1238,12817,14055,-728,460,33018,2308,33844.0,15537.0,49381,578449,283560.0
7,6,2020-11-25T17:00:00,8,Emilia-Romagna,2763,249,3012,69514,72526,1182,2130,34681,5493,70428.0,42272.0,112700,2040254,1048727.0
8,7,2020-11-25T17:00:00,9,Toscana,1754,286,2040,47240,49280,-1426,986,46318,2378,75858.0,22118.0,97976,1494998,928848.0
9,8,2020-11-25T17:00:00,10,Umbria,361,70,431,9506,9937,-260,386,12186,355,6314.0,16164.0,22478,399810,219242.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-26T17:00:00,1,Piemonte,4992,403,5395,73011,78406,485,2751,74723,5875,67389.0,91615.0,159004,1490715,933365.0,4341375
1,2020-11-26T17:00:00,2,Valle d'Aosta,128,17,145,1468,1613,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0,125501
2,2020-11-26T17:00:00,3,Lombardia,7996,934,8930,131471,140401,2372,5697,231042,21212,290349.0,102306.0,392655,3949040,2304509.0,10103969
3,2020-11-26T17:00:00,4,Trentino-Alto Adige,851,83,934,12878,13812,-235,589,22161,1124,29832.0,7265.0,37097,666100,277978.0,1074819
4,2020-11-26T17:00:00,5,Veneto,2447,304,2751,72401,75152,14,3980,55403,3501,34933.0,99123.0,134056,2734036,1045655.0,4907704
5,2020-11-26T17:00:00,6,Friuli Venezia Giulia,599,55,654,13168,13822,286,814,12536,741,23010.0,4089.0,27099,687612,270753.0,1211357
6,2020-11-26T17:00:00,7,Liguria,1075,114,1189,12303,13492,-563,570,34137,2322,34280.0,15671.0,49951,584154,285519.0,1543127
7,2020-11-26T17:00:00,8,Emilia-Romagna,2679,258,2937,68535,71472,-1054,2157,37835,5548,71624.0,43231.0,114855,2057518,1055046.0,4467118
8,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0,3722729
9,2020-11-26T17:00:00,10,Umbria,349,68,417,9017,9434,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-26T17:00:00,1,Piemonte,4992,403,5395,73011,78406,485,2751,74723,5875,67389.0,91615.0,159004,1490715,933365.0,4341375,575
1,2020-11-26T17:00:00,2,Valle d'Aosta,128,17,145,1468,1613,-70,44,4328,302,5561.0,682.0,6243,57624,32671.0,125501,20
2,2020-11-26T17:00:00,3,Lombardia,7996,934,8930,131471,140401,2372,5697,231042,21212,290349.0,102306.0,392655,3949040,2304509.0,10103969,1036
3,2020-11-26T17:00:00,4,Trentino-Alto Adige,851,83,934,12878,13812,-235,589,22161,1124,29832.0,7265.0,37097,666100,277978.0,1074819,106
4,2020-11-26T17:00:00,5,Veneto,2447,304,2751,72401,75152,14,3980,55403,3501,34933.0,99123.0,134056,2734036,1045655.0,4907704,1016
5,2020-11-26T17:00:00,6,Friuli Venezia Giulia,599,55,654,13168,13822,286,814,12536,741,23010.0,4089.0,27099,687612,270753.0,1211357,180
6,2020-11-26T17:00:00,7,Liguria,1075,114,1189,12303,13492,-563,570,34137,2322,34280.0,15671.0,49951,584154,285519.0,1543127,209
7,2020-11-26T17:00:00,8,Emilia-Romagna,2679,258,2937,68535,71472,-1054,2157,37835,5548,71624.0,43231.0,114855,2057518,1055046.0,4467118,634
8,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,48742,2450,76637.0,22690.0,99327,1511997,935992.0,3722729,523
9,2020-11-26T17:00:00,10,Umbria,349,68,417,9017,9434,-503,343,13018,369,6394.0,16427.0,22821,404601,220882.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-26T17:00:00,3,Lombardia,7996,934,8930,131471,140401,2372,5697,...,90.15,2498,524,44231,15288.0,207,3118,12.88,0.05638,3.89
1,2020-11-26T17:00:00,5,Veneto,2447,304,2751,72401,75152,14,3980,...,29.92,-10,1320,19146,5885.0,72,3894,20.79,0.08110,2.73
2,2020-11-26T17:00:00,15,Campania,2212,200,2412,100972,103384,1236,3008,...,39.60,1298,193,23761,11352.0,49,1723,12.66,0.05199,2.52
3,2020-11-26T17:00:00,1,Piemonte,4992,403,5395,73011,78406,485,2751,...,70.09,588,-127,22418,9485.0,72,2194,12.27,0.06337,3.66
4,2020-11-26T17:00:00,12,Lazio,3408,352,3760,83543,87303,738,2260,...,41.56,675,158,25908,22511.0,69,1453,8.72,0.03853,1.91
5,2020-11-26T17:00:00,8,Emilia-Romagna,2679,258,2937,68535,71472,-1054,2157,...,40.69,-979,27,17264,6319.0,55,3154,12.49,0.04829,2.57
6,2020-11-26T17:00:00,19,Sicilia,1545,253,1798,36710,38508,188,1768,...,43.03,214,451,11500,7202.0,49,1531,15.37,0.03558,1.18
7,2020-11-26T17:00:00,16,Puglia,1577,211,1788,33191,34979,908,1436,...,57.18,813,-75,9612,5613.0,52,476,14.94,0.03583,1.22
8,2020-11-26T17:00:00,9,Toscana,1712,281,1993,46142,48135,-1145,1351,...,53.73,-1098,365,16999,7144.0,72,2424,7.95,0.03629,2.67
9,2020-11-26T17:00:00,6,Friuli Venezia Giulia,599,55,654,13168,13822,286,814,...,30.56,274,118,7626,1903.0,26,502,10.67,0.06720,2.24


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,934,102,1036,90.15
1,Valle d'Aosta,17,3,20,85.00
2,Trentino-Alto Adige,83,23,106,78.30
3,Piemonte,403,172,575,70.09
4,Marche,90,53,143,62.94
5,Umbria,68,49,117,58.12
6,Puglia,211,158,369,57.18
7,Liguria,114,95,209,54.55
8,Toscana,281,242,523,53.73
9,Abruzzo,74,74,148,50.00


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,2020-11-24T17:00:00,22,P.A. Trento,436,42,478,1994,2472,-46,143,11297,599,7423.0,6945.0,14368,356152,129600.0
550,2020-11-25T17:00:00,21,P.A. Bolzano,442,41,483,11117,11600,-56,341,9863,498,21961.0,0.0,21961,300122,146006.0
551,2020-11-25T17:00:00,22,P.A. Trento,421,41,462,1985,2447,-25,179,11494,606,7483.0,7064.0,14547,359553,130165.0
552,2020-11-26T17:00:00,21,P.A. Bolzano,434,40,474,10976,11450,-150,292,10294,509,22253.0,0.0,22253,302226,146515.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,...,8.0,6424.0,1844.0,-221.0,8.30,6.72,10.80,37.64,0.00,4.827
273,2020-11-23T17:00:00,4,Trentino-Alto Adige,906,81,987,13146,14133,-33,353,...,12.0,2947.0,960.0,-68.0,8.21,6.98,11.98,36.77,2.53,2.492
274,2020-11-24T17:00:00,4,Trentino-Alto Adige,895,82,977,13151,14128,-5,303,...,30.0,3649.0,1169.0,5.0,8.39,6.92,8.30,25.92,1.23,2.144
275,2020-11-25T17:00:00,4,Trentino-Alto Adige,863,82,945,13102,14047,-81,520,...,15.0,6135.0,1176.0,-49.0,8.68,6.73,8.48,44.22,0.00,3.681


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
